In [19]:
import pandas as pd
import datetime

data = pd.read_csv('access.log.txt', header=None, sep=' ')
data = data.rename(columns={0: 'ip', 1: 'dash1', 2: 'dash2', 3: 'timestamp', 4: 'timezone', 5: 'http_method',
                            6: 'endpoint', 7: 'http_version', 8: 'status_code', 9: 'response_size', 10: 'dash3',
                            11: 'user_agent'})
data['timestamp'] = data['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '[%d/%b/%Y:%H:%M:%S').strftime('%d/%b/%Y:%H:%M:%S'))
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%d/%b/%Y:%H:%M:%S')
data = data.drop(columns=['dash1', 'dash2', 'timezone', 'http_version'])
data['department'] = data['response_size'].str.extract('/department/(\w+)')
data['product'] = data['response_size'].str.extract('/product/([\w%]+)')
# data['weekday'] = data['timestamp'].dt.weekday_name
data['hour'] = data['timestamp'].dt.hour
# data['response_size'] = data['response_size'].astype(int)

import re

def extract_department_product(endpoint):
    pattern = r'^/department/(?P<department>[^/]+)/.*?product/(?P<product>[^/ ]+)'
    match = re.match(pattern, endpoint)
    if match:
        department = match.group('department')
        product = match.group('product')
    else:
        department = None
        product = None
    return department, product

data[['department', 'product']] = data['http_method'].apply(extract_department_product, 1)
data.drop('http_method', axis=1, inplace=True)


data

ValueError: Columns must be same length as key

In [16]:
data.http_method[1]

"GET /department/apparel/category/featured%20shops/product/adidas%20Kids'%20RG%20III%20Mid%20Football%20Cleat HTTP/1.1"